In [57]:
from pyscf  import gto, dft, scf, lib, mp, cc
import scipy as sp
import numpy as np
from copy import deepcopy

In [67]:
H_bond=0.74
R= 1.2 # distance of H2 --- R --- H2
n_active_atoms=2

geometry = f'H 0, 0, {-H_bond}; H 0, 0, {0}; H 0, 0, {R}; H 0, 0, {R+H_bond}'


basis_set = 'STO-6G'
low_level_xc_functional = 'lda,vwn'
high_level_xc_functional = 'b3lyp'

low_level_method = 'RKS'
high_level_method = 'CC'

In [59]:
geometry

'H 0, 0, -0.74; H 0, 0, 0; H 0, 0, 1.2; H 0, 0, 1.94'

In [68]:
molecule = gto.Mole(atom=geometry,
               basis=basis_set,
               charge=0,
               spin=0)

print(molecule.atom)
print('')
molecule.build()

H 0, 0, -0.74; H 0, 0, 0; H 0, 0, 1.2; H 0, 0, 1.94



In [69]:
molecule.verbose = 1

# Run low level global calc

In [73]:
low_level_full_scf = scf.RKS(molecule) # DFT
# low_level_full_scf = scf.RHF(molecule) # HF
low_level_full_scf.verbose = 1
low_level_full_scf.conv_tol = 1e-6
low_level_full_scf.xc = low_l evel_xc_functional
low_level_full_scf.kernel()

-2.246994941228569

In [74]:
low_level_full_V_xc

array([[1.22105316, 0.73151519, 0.13352854, 0.04962388],
       [0.73151519, 1.40647454, 0.50483241, 0.13352854],
       [0.13352854, 0.50483241, 1.40647454, 0.73151519],
       [0.04962388, 0.13352854, 0.73151519, 1.22105316]])

In [75]:
low_level_full_V_xc = low_level_full_scf.get_veff()
low_level_full_E_xc = low_level_full_scf.get_veff().exc

In [76]:
double_occ =  sum(low_level_full_scf.mo_occ == 2)

occupied_orbitals_full_system = low_level_full_scf.mo_coeff[:, :double_occ]

J_low_level_full, K_low_level_full = low_level_full_scf.get_jk()

S_ao_overalp_low_level_full = low_level_full_scf.get_ovlp()

In [77]:
n_active_ao = molecule.aoslice_nr_by_atom()[n_active_atoms-1][3] # gets length of active list of atoms
n_active_ao

2

In [78]:
occupied_orbitals_full_system

array([[ 0.28801803, -0.48182558],
       [ 0.40601711, -0.37485039],
       [ 0.40601711,  0.37485039],
       [ 0.28801803,  0.48182558]])

# Either use SPADE or Huzinaga

In [103]:
partitioning_method = 'huzinaga'#
# partitioning_method = 'SPADE'

if partitioning_method.lower() == 'spade':
    s_half = sp.linalg.fractional_matrix_power(S_ao_overalp_low_level_full, 0.5)
    ortho_orbitals = (s_half@ occupied_orbitals_full_system)[:n_active_ao, :]
    
    U, singular_vals, rot_matrix = np.linalg.svd(ortho_orbitals, full_matrices=True)
    
    
    delta_s = [-(singular_vals[i+1]-singular_vals[i]) for i in range(len(singular_vals)-1)]
    n_active_MOs = np.argpartition(delta_s, -1)[-1]+1
    n_env_MOs = singular_vals.shape[0] - n_active_ao
    
elif (partitioning_method.lower() == 'huzinaga') or (partitioning_method.lower() == 'mu'):
    
    ortho_orbitals = occupied_orbitals_full_system
    U, singular_vals, rot_matrix = np.linalg.svd(ortho_orbitals, full_matrices=True)
    
    n_active_MOs = n_active_ao
    n_env_MOs = singular_vals.shape[0] - n_active_ao
    
else:
    raise ValueError(f'unknown partitioning method: {partitioning_method}')

In [102]:
C = lo.orth_ao(test_scf, 'nao') # matrix of AO to localized orbital coefficients

# Define active and environment e- ensities

In [106]:
C

array([[ 1.26145289, -0.49117208,  0.07496671, -0.02386852],
       [-0.5139852 ,  1.34962289, -0.30745091,  0.07844863],
       [ 0.07844863, -0.30745091,  1.34962289, -0.5139852 ],
       [-0.02386852,  0.07496671, -0.49117208,  1.26145289]])

In [126]:
orbs = low_level_full_scf.mo_coeff
U, singular_vals, rot_matrix = np.linalg.svd(orbs, full_matrices=True)
orbs @ U

array([[-1.25782292, -0.49502625, -0.10001484,  0.03912392],
       [ 0.50260038,  1.34849763,  0.31777666, -0.11961586],
       [-0.05045658, -0.29164232, -1.35285179,  0.51820258],
       [ 0.01374219,  0.03552105,  0.49637971, -1.26129287]])

In [110]:
active_orbitals = occupied_orbitals_full_system @ rot_matrix.T[:,:n_active_MOs]
environ_orbitals = occupied_orbitals_full_system @ rot_matrix.T[:,n_active_MOs:]

gamma_active =  2 * np.matmul(active_orbitals,active_orbitals.T)
gamma_environ = 2 * np.matmul(environ_orbitals,environ_orbitals.T)

In [30]:
gamma_active

array([[ 0.59265918,  0.60114585,  0.0239935 , -0.1492015 ],
       [ 0.60114585,  0.60975405,  0.02433708, -0.15133801],
       [ 0.0239935 ,  0.02433708,  0.00097136, -0.00604035],
       [-0.1492015 , -0.15133801, -0.00604035,  0.03756137]])

## Get closed shall active energy

In [34]:
j_act = low_level_full_scf.get_j(dm=gamma_active) # also contains k!
k_act = np.zeros_like(j_act)

two_elec_term_act = low_level_full_scf.get_veff(dm=gamma_active)
e_xc_act = two_elec_term_act.exc
v_xc_act = two_elec_term_act - j_act

E_act = np.einsum('ij,ij', gamma_active, low_level_full_scf.get_hcore() + j_act/2) + e_xc_act
E_act

-2.9926389899128596

In [35]:
e_xc_act

-0.67703917909991

## Get closed shall environment energy

In [36]:
j_env = low_level_full_scf.get_j(dm=gamma_environ) # also contains k!
k_env = np.zeros_like(j_env)

two_elec_term_env = low_level_full_scf.get_veff(dm=gamma_environ)
e_xc_env = two_elec_term_env.exc
v_xc_env = two_elec_term_env - j_env

E_env = np.einsum('ij,ij', gamma_environ, low_level_full_scf.get_hcore() + j_env/2) + e_xc_env
E_env

-2.9926389899128587

# get cross system terms

In [37]:
J_cross = 0.5 * (np.einsum('ij,ij', gamma_active, j_env) + np.einsum('ij,ij', gamma_environ, j_act))
K_cross = 0 # pyscf includes in J!

xc_cross = low_level_full_E_xc - e_xc_act- e_xc_env

two_elec_cross = J_cross + K_cross + xc_cross

1.1240956517435132

# Define embedded Hamiltonian

In [39]:
projection_method = 'mu'

if projection_method.lower() == 'huzinaga':
    F_gb_S = low_level_full_scf.get_fock() @ gamma_environ @ S_ao_overalp_low_level_full
    Projector = -0.5*(F_gb_S + F_gb_S.T)
    
elif projection_method.lower() == 'mu':
    Projector = 1e6 * S_ao_overalp_low_level_full @ gamma_environ @ S_ao_overalp_low_level_full
else:
    raise ValueError(f'unknown partitioning method: {partitioning_method}')

In [40]:
h_core = low_level_full_scf.get_hcore()
v_embed = j_env - k_env + low_level_full_V_xc - v_xc_act + Projector


In [47]:
low_level_full_V_xc

NPArrayWithTag([[1.3676255 , 0.96720547, 0.19268363, 0.05419421],
                [0.96720547, 1.5639534 , 0.59270491, 0.19268363],
                [0.19268363, 0.59270491, 1.5639534 , 0.96720547],
                [0.05419421, 0.19268363, 0.96720547, 1.3676255 ]])

In [41]:
v_embed

array([[   1663.76090846,  -12528.26415627,  -51980.91409925,
         -52243.4664529 ],
       [ -12528.26415627,   94496.03953891,  392014.16282257,
         393991.66536417],
       [ -51980.91409925,  392014.16282257, 1626300.21049547,
        1634505.87923403],
       [ -52243.4664529 ,  393991.66536417, 1634505.87923403,
        1642754.97264509]])

# run embedded scf

In [44]:
molecule_EMBED = deepcopy(molecule)
# molecule_EMBED.nelectrons = 2*n_active_MOs

high_level_full_scf_EMBED = scf.RHF(molecule)
high_level_full_scf_EMBED.verbose = 1
high_level_full_scf_EMBED.conv_tol = 1e-6


high_level_full_scf_EMBED.get_hcore = lambda *args: h_core + v_embed
high_level_full_scf_EMBED.kernel()

9.725887666890053

In [ ]:
double_occ_EMBED =  sum(high_level_full_scf_EMBED.mo_occ == 2)
occupied_orbitals_EMBED = high_level_full_scf_EMBED.mo_coeff[:, :double_occ_EMBED]

density_embed = 2 * occupied_orbitals_EMBED @ occupied_orbitals_EMBED.T

In [ ]:
J_embed, K_embed = high_level_full_scf_EMBED.get_jk()

In [ ]:
E_active_EMBED = np.einsum('ij, ij', density_embed, h_core + 0.5*J_embed - 0.25*K_embed)

correction = np.einsum('ij, ij', v_embed, density_embed-gamma_active)

In [ ]:
E_embedded = E_active_EMBED + E_env + two_elec_cross + high_level_full_scf_EMBED.energy_nuc() + correction 
E_embedded

## Wavefunction Calc

In [ ]:
shift = mol_low_level.nao - n_env_MOs
shift

In [ ]:
if projection_method.lower() == 'mu':
    shift = mol_low_level.nao - n_env_MOs
    # freeze mu shifted orbitals@
    frozen_orbs = [i for i in range(shift, mol_low_level.nao)]

elif projection_method.lower() == 'huzinaga':
    # not sure if frozen orbs needed here... as projected out.
    projected_out = mol_low_level.nao - n_env_MOs
    frozen_orbs = [i for i in range(projected_out, mol_low_level.nao)]
    
else:
    # no frozen
    pass

CC_scf = cc.CCSD(high_level_full_scf_EMBED)#.set(frozen = frozen_orbs)
e_correlation, t1, t2 = CC_scf.kernel()

# print(CC_scf.e_tot)

E_WF_embedded = E_embedded + e_correlation
E_WF_embedded

In [ ]:
low_level_full_scf = scf.RHF(molecule)
low_level_full_scf.verbose = 1
low_level_full_scf.conv_tol = 1e-6
low_level_full_scf.kernel()
CC_scf = cc.CCSD(low_level_full_scf)#.set(frozen = frozen_orbs)
e_correlation, t1, t2 = CC_scf.kernel()

print(CC_scf.e_tot == low_level_full_scf.e_tot + e_correlation)
CC_scf.e_tot

In [84]:
test_scf = scf.RKS(molecule) # DFT
test_scf.verbose = 1
test_scf.conv_tol = 1e-6
test_scf.xc = low_level_xc_functional
test_scf.kernel()

-2.246994941228569

In [88]:
# localise orbitals
from pyscf import lo

C = lo.orth_ao(test_scf, 'nao') # matrix of AO to localized orbital coefficients

array([[ 1.26145289, -0.49117208,  0.07496671, -0.02386852],
       [-0.5139852 ,  1.34962289, -0.30745091,  0.07844863],
       [ 0.07844863, -0.30745091,  1.34962289, -0.5139852 ],
       [-0.02386852,  0.07496671, -0.49117208,  1.26145289]])

In [101]:
mo  = np.linalg.solve(C, test_scf.mo_coeff)
# mo = C.T @ test_scf.get_ovlp() @ test_scf.mo_coeff

dm = test_scf.make_rdm1(mo, test_scf.mo_occ)

test_scf.mulliken_pop(molecule, dm, np.eye(molecule.nao_nr()))

 ** Mulliken pop  **
pop of  0 H 1s        0.99262
pop of  1 H 1s        1.00738
pop of  2 H 1s        1.00738
pop of  3 H 1s        0.99262
 ** Mulliken atomic charges  **
charge of  0H =      0.00738
charge of  1H =     -0.00738
charge of  2H =     -0.00738
charge of  3H =      0.00738


(array([0.99262285, 1.00737715, 1.00737715, 0.99262285]),
 array([ 0.00737715, -0.00737715, -0.00737715,  0.00737715]))